In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_fscore_support, classification_report
from scipy import signal
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import os

In [2]:
dataframes = []
directory_path = os.path.join(os.getcwd(), 'tdcsfog_data')
print(directory_path)

/Users/Mateo/FOGproject/notebooks/tdcsfog_data


In [16]:
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        # Full path to the file
        file_path = os.path.join(directory_path, filename)
        # Extract patient ID from the file name (before '.csv')
        patient_id = filename.split('.')[0]
        # Read the CSV file
        df = pd.read_csv(file_path)
        # Add patient ID column
        df['PatientID'] = patient_id
        # Append to the list of dataframes
        dataframes.append(df)

# Combine all dataframes into one
tdcsfog_df = pd.concat(dataframes, ignore_index=True)

tdcsfog_df.head(10)

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,PatientID
0,0,-9.802023,-0.436305,-1.606585,0,0,0,58b1cb50ad
1,1,-9.802450,-0.443045,-1.599153,0,0,0,58b1cb50ad
2,2,-9.803275,-0.439261,-1.602125,0,0,0,58b1cb50ad
3,3,-9.801431,-0.436956,-1.597653,0,0,0,58b1cb50ad
4,4,-9.800925,-0.434741,-1.600627,0,0,0,58b1cb50ad
5,5,-9.800947,-0.433229,-1.599137,0,0,0,58b1cb50ad
6,6,-9.797858,-0.436299,-1.599136,0,0,0,58b1cb50ad
7,7,-9.799281,-0.437771,-1.597652,0,0,0,58b1cb50ad
8,8,-9.803057,-0.434677,-1.596909,0,0,0,58b1cb50ad
9,9,-9.797640,-0.431715,-1.593920,0,0,0,58b1cb50ad


In [17]:
class FOGPreprocessor:
    def __init__(self, sampling_rate=128, window_size=3, stride=2):
        self.sampling_rate = sampling_rate
        self.window_size = window_size
        self.stride = stride
        self.scaler = StandardScaler()
        
    def butter_bandpass_filter(self, data, lowcut=0.25, highcut=10, order=4):
        nyquist = 0.5 * self.sampling_rate
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = signal.butter(order, [low, high], btype='band')
        return signal.filtfilt(b, a, data)
    
    def create_windows(self, df, label_column):
        """
        Create overlapping windows for a specific label
        
        Args:
            df: DataFrame with sensor data
            label_column: The specific FOG type to process ('StartHesitation', 'Turn', or 'Walking')
        """
        window_samples = self.window_size * self.sampling_rate
        stride_samples = self.stride * self.sampling_rate
        
        # Apply bandpass filter to acceleration columns
        acc_columns = ['AccV', 'AccML', 'AccAP']
        filtered_data = {}
        for col in acc_columns:
            filtered_data[col] = self.butter_bandpass_filter(df[col].values)
        
        # Create DataFrame with filtered data
        filtered_df = pd.DataFrame(filtered_data)
        
        # Standardize the features
        filtered_df = pd.DataFrame(
            self.scaler.fit_transform(filtered_df),
            columns=filtered_df.columns
        )
        
        # Create sliding windows
        n_samples = len(df)
        n_windows = ((n_samples - window_samples) // stride_samples) + 1
        n_features = len(filtered_df.columns)
        
        X = np.zeros((n_windows, window_samples, n_features))
        y = np.zeros(n_windows)
        
        for i in range(n_windows):
            start_idx = i * stride_samples
            end_idx = start_idx + window_samples
            
            X[i] = filtered_df.iloc[start_idx:end_idx].values
            # Get labels for the specific FOG type
            window_labels = df.iloc[start_idx:end_idx][label_column]
            y[i] = 1 if window_labels.sum() > 0 else 0
        
        return X, y

In [18]:
class FOGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [24]:
class BiGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.3):
        super(BiGRUModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.batch_norm = nn.BatchNorm1d(input_size)
        
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.batch_norm(x.transpose(1, 2)).transpose(1, 2)
        gru_out, _ = self.gru(x)
        attention_weights = self.attention(gru_out)
        context = torch.sum(attention_weights * gru_out, dim=1)
        out = self.fc(context)
        return out.squeeze()

In [25]:
class FOGTrainer:
    def __init__(self, model, device, fog_type, learning_rate=0.001):
        self.model = model.to(device)
        self.device = device
        self.fog_type = fog_type
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(batch_X)
            loss = self.criterion(outputs, batch_y)
            
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(train_loader)
    
    def evaluate(self, val_loader):
        self.model.eval()
        total_loss = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                total_loss += loss.item()
                
                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        pred_labels = (all_preds > 0.5).astype(int)
        
        # Calculate metrics with zero division handling
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, pred_labels, 
                                                                average='binary',
                                                                zero_division=0)
        
        # Handle case where all predictions are the same class
        try:
            auc = roc_auc_score(all_labels, all_preds)
        except:
            auc = 0.0
            
        conf_matrix = confusion_matrix(all_labels, pred_labels)
        
        # Get detailed classification report with zero division handling
        report = classification_report(all_labels, pred_labels, 
                                    output_dict=True,
                                    zero_division=0)
        
        return {
            'loss': total_loss / len(val_loader),
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'confusion_matrix': conf_matrix,
            'classification_report': report
        }
    
    def train(self, train_loader, val_loader, num_epochs=50, patience=5):
        best_val_f1 = 0
        patience_counter = 0
        best_metrics = None
        
        for epoch in range(num_epochs):
            train_loss = self.train_epoch(train_loader)
            val_metrics = self.evaluate(val_loader)
            
            print(f'\nEpoch {epoch+1}/{num_epochs} - {self.fog_type}:')
            print(f'Train Loss: {train_loss:.4f}')
            print(f'Val Loss: {val_metrics["loss"]:.4f}')
            report = val_metrics['classification_report']
            
            print(f'Val AUC: {val_metrics["auc"]:.4f}')
            print('\nDetailed Metrics:')
            print(f"Weighted Precision: {report['weighted avg']['precision']:.4f}")
            print(f"Weighted Recall: {report['weighted avg']['recall']:.4f}")
            print(f"Weighted F1-score: {report['weighted avg']['f1-score']:.4f}")
            print(f"Support: {report['weighted avg']['support']}")
            
            print('\nClass-wise Metrics:')
            class_metrics = val_metrics['classification_report']
            try:
                print(f"Class 0 (No FOG) - Precision: {class_metrics['0']['precision']:.4f}, "
                      f"Recall: {class_metrics['0']['recall']:.4f}, F1: {class_metrics['0']['f1-score']:.4f}, "
                      f"Support: {class_metrics['0']['support']}")
            except KeyError:
                print("No predictions for Class 0 (No FOG)")
                
            try:
                print(f"Class 1 (FOG) - Precision: {class_metrics['1']['precision']:.4f}, "
                      f"Recall: {class_metrics['1']['recall']:.4f}, F1: {class_metrics['1']['f1-score']:.4f}, "
                      f"Support: {class_metrics['1']['support']}")
            except KeyError:
                print("No predictions for Class 1 (FOG)")
                
            conf_matrix = val_metrics['confusion_matrix']
            # Calculate class distribution from confusion matrix
            print("\nClass Distribution:")
            true_dist = conf_matrix.sum(axis=1)
            pred_dist = conf_matrix.sum(axis=0)
            print(f"Labels distribution: {dict(enumerate(true_dist))}")
            print(f"Predictions distribution: {dict(enumerate(pred_dist))}")
            print('Confusion Matrix:')
            print(val_metrics['confusion_matrix'])
            
            if val_metrics['f1'] > best_val_f1:
                best_val_f1 = val_metrics['f1']
                best_metrics = val_metrics
                patience_counter = 0
                torch.save(self.model.state_dict(), f'best_model_{self.fog_type}.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'\nEarly stopping triggered for {self.fog_type}')
                    print(f'Best metrics achieved:')
                    print(f"F1 Score: {best_metrics['f1']:.4f}")
                    print(f"Precision: {best_metrics['precision']:.4f}")
                    print(f"Recall: {best_metrics['recall']:.4f}")
                    print(f"AUC: {best_metrics['auc']:.4f}")
                    break
            
            print('-' * 50)

def train_fog_models(df):
    """Train separate models for each FOG type"""
    preprocessor = FOGPreprocessor()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    fog_types = ['StartHesitation', 'Turn', 'Walking']
    
    results = {}
    for fog_type in fog_types:
        print(f"\nTraining model for {fog_type}")
        
        # Preprocess data for this FOG type
        X, y = preprocessor.create_windows(df, fog_type)
        
        # Create datasets
        dataset = FOGDataset(X, y)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        # Initialize model and trainer
        model = BiGRUModel(input_size=X.shape[2])
        trainer = FOGTrainer(model, device, fog_type)
        
        # Train model
        trainer.train(train_loader, val_loader)
        
        # Store results
        results[fog_type] = {
            'model': model,
            'metrics': trainer.evaluate(val_loader)
        }
    
    return results

In [26]:
# Assuming you have your data in tdcsfog_df
results = train_fog_models(tdcsfog_df)

# Print final results for each FOG type
for fog_type, result in results.items():
    print(f"\nFinal results for {fog_type}:")
    metrics = result['metrics']
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print("Confusion Matrix:")
    print(metrics['confusion_matrix'])


Training model for StartHesitation



Epoch 1/50 - StartHesitation:
Train Loss: 0.1812
Val Loss: 0.1431
Val AUC: 0.8611

Detailed Metrics:
Weighted Precision: 0.9097
Weighted Recall: 0.9538
Weighted F1-score: 0.9312
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5263, 1: 255}
Predictions distribution: {0: 5518, 1: 0}
Confusion Matrix:
[[5263    0]
 [ 255    0]]
--------------------------------------------------



Epoch 2/50 - StartHesitation:
Train Loss: 0.1503
Val Loss: 0.1333
Val AUC: 0.8805

Detailed Metrics:
Weighted Precision: 0.9385
Weighted Recall: 0.9543
Weighted F1-score: 0.9349
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5263, 1: 255}
Predictions distribution: {0: 5501, 1: 17}
Confusion Matrix:
[[5256    7]
 [ 245   10]]
--------------------------------------------------



Epoch 3/50 - StartHesitation:
Train Loss: 0.1431
Val Loss: 0.1275
Val AUC: 0.8842

Detailed Metrics:
Weighted Precision: 0.9471
Weighted Recall: 0.9570
Weighted F1-score: 0.9426
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5263, 1: 255}
Predictions distribution: {0: 5472, 1: 46}
Confusion Matrix:
[[5249   14]
 [ 223   32]]
--------------------------------------------------



Epoch 4/50 - StartHesitation:
Train Loss: 0.1351
Val Loss: 0.1274
Val AUC: 0.8936

Detailed Metrics:
Weighted Precision: 0.9499
Weighted Recall: 0.9585
Weighted F1-score: 0.9460
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5263, 1: 255}
Predictions distribution: {0: 5458, 1: 60}
Confusion Matrix:
[[5246   17]
 [ 212   43]]
--------------------------------------------------


...


Epoch 22/50 - Walking:
Train Loss: 0.0397
Val Loss: 0.1001
Val AUC: 0.9379

Detailed Metrics:
Weighted Precision: 0.9662
Weighted Recall: 0.9670
Weighted F1-score: 0.9666
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5326, 1: 192}
Predictions distribution: {0: 5336, 1: 182}
Confusion Matrix:
[[5240   86]
 [  96   96]]
--------------------------------------------------



Epoch 23/50 - Walking:
Train Loss: 0.0366
Val Loss: 0.1128
Val AUC: 0.9310

Detailed Metrics:
Weighted Precision: 0.9673
Weighted Recall: 0.9717
Weighted F1-score: 0.9683
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5326, 1: 192}
Predictions distribution: {0: 5402, 1: 116}
Confusion Matrix:
[[5286   40]
 [ 116   76]]
--------------------------------------------------



Epoch 24/50 - Walking:
Train Loss: 0.0345
Val Loss: 0.1105
Val AUC: 0.9284

Detailed Metrics:
Weighted Precision: 0.9658
Weighted Recall: 0.9659
Weighted F1-score: 0.9658
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5326, 1: 192}
Predictions distribution: {0: 5328, 1: 190}
Confusion Matrix:
[[5233   93]
 [  95   97]]
--------------------------------------------------



Epoch 25/50 - Walking:
Train Loss: 0.0330
Val Loss: 0.1260
Val AUC: 0.9281

Detailed Metrics:
Weighted Precision: 0.9658
Weighted Recall: 0.9690
Weighted F1-score: 0.9671
Support: 5518.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 5326, 1: 192}
Predictions distribution: {0: 5369, 1: 149}
Confusion Matrix:
[[5262   64]
 [ 107   85]]

Early stopping triggered for Walking
Best metrics achieved:
F1 Score: 0.5145
Precision: 0.6723
Recall: 0.4167
AUC: 0.9376



Final results for StartHesitation:
Precision: 0.7868
Recall: 0.4196
F1 Score: 0.5473
AUC: 0.9313
Confusion Matrix:
[[5234   29]
 [ 148  107]]

Final results for Turn:
Precision: 0.7685
Recall: 0.7988
F1 Score: 0.7834
AUC: 0.9385
Confusion Matrix:
[[3600  372]
 [ 311 1235]]

Final results for Walking:
Precision: 0.5705
Recall: 0.4427
F1 Score: 0.4985
AUC: 0.9281
Confusion Matrix:
[[5262   64]
 [ 107   85]]


In [27]:
dataframes = []
directory_path = os.path.join(os.getcwd(), 'defog_data')
print(directory_path)

/Users/Mateo/FOGproject/notebooks/defog_data


In [28]:
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        # Full path to the file
        file_path = os.path.join(directory_path, filename)
        # Extract patient ID from the file name (before '.csv')
        patient_id = filename.split('.')[0]
        # Read the CSV file
        df = pd.read_csv(file_path)
        # Add patient ID column
        df['PatientID'] = patient_id
        # Append to the list of dataframes
        dataframes.append(df)

# Combine all dataframes into one
defog_df = pd.concat(dataframes, ignore_index=True)

defog_df.head(10)

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,PatientID
0,0,-1.035290,0.121326,-0.183682,0,0,0,False,False,3f970065e5
1,1,-1.037887,0.116788,-0.189828,0,0,0,False,False,3f970065e5
2,2,-1.014818,0.125752,-0.174294,0,0,0,False,False,3f970065e5
3,3,-1.000146,0.128355,-0.156082,0,0,0,False,False,3f970065e5
4,4,-0.985992,0.117344,-0.141497,0,0,0,False,False,3f970065e5
5,5,-0.977788,0.105946,-0.124575,0,0,0,False,False,3f970065e5
6,6,-0.979650,0.096025,-0.113552,0,0,0,False,False,3f970065e5
7,7,-0.994607,0.086037,-0.122174,0,0,0,False,False,3f970065e5
8,8,-1.016418,0.075515,-0.131091,0,0,0,False,False,3f970065e5
9,9,-1.017296,0.060953,-0.127201,0,0,0,False,False,3f970065e5


In [29]:
results = train_fog_models(defog_df)

# Print final results for each FOG type
for fog_type, result in results.items():
    print(f"\nFinal results for {fog_type}:")
    metrics = result['metrics']
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print("Confusion Matrix:")
    print(metrics['confusion_matrix'])


Training model for StartHesitation



Epoch 1/50 - StartHesitation:
Train Loss: 0.0081
Val Loss: 0.0034
Val AUC: 0.7847

Detailed Metrics:
Weighted Precision: 0.9994
Weighted Recall: 0.9997
Weighted F1-score: 0.9996
Support: 10567.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 10564, 1: 3}
Predictions distribution: {0: 10567, 1: 0}
Confusion Matrix:
[[10564     0]
 [    3     0]]
--------------------------------------------------



Epoch 2/50 - StartHesitation:
Train Loss: 0.0025
Val Loss: 0.0032
Val AUC: 0.7217

Detailed Metrics:
Weighted Precision: 0.9994
Weighted Recall: 0.9997
Weighted F1-score: 0.9996
Support: 10567.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 10564, 1: 3}
Predictions distribution: {0: 10567, 1: 0}
Confusion Matrix:
[[10564     0]
 [    3     0]]
--------------------------------------------------



Epoch 3/50 - StartHesitation:
Train Loss: 0.0022
Val Loss: 0.0030
Val AUC: 0.7793

Detailed Metrics:
Weighted Precision: 0.9994
Weighted Recall: 0.9997
Weighted F1-score: 0.9996
Support: 10567.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 10564, 1: 3}
Predictions distribution: {0: 10567, 1: 0}
Confusion Matrix:
[[10564     0]
 [    3     0]]
--------------------------------------------------



Epoch 4/50 - StartHesitation:
Train Loss: 0.0025
Val Loss: 0.0034
Val AUC: 0.7600

Detailed Metrics:
Weighted Precision: 0.9994
Weighted Recall: 0.9997
Weighted F1-score: 0.9996
Support: 10567.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 10564, 1: 3}
Predictions distribution: {0: 10567, 1: 0}
Confusion Matrix:
[[10564     0]
 [    3     0]]
--------------------------------------------------



Epoch 5/50 - StartHesitation:
Train Loss: 0.0027
Val Loss: 0.0026
Val AUC: 0.8537

Detailed Metrics:
Weighted Precision: 0.9994
Weighted Recall: 0.9997
Weighted F1-score: 0.9996
Support: 10567.0

Class-wise Metrics:
No predictions for Class 0 (No FOG)
No predictions for Class 1 (FOG)

Class Distribution:
Labels distribution: {0: 10564, 1: 3}
Predictions distribution: {0: 10567, 1: 0}
Confusion Matrix:
[[10564     0]
 [    3     0]]

Early stopping triggered for StartHesitation
Best metrics achieved:


TypeError: 'NoneType' object is not subscriptable